### Développement d’un pipeline de calcul du TMB

#### Projet 4BiM, 2021

#### Auteurs : Marie Casimir, Loup Petitjean et Nicolas Mendiboure
#### Encadrantes Innate-Pharma : Sabrina Carpentier et Luciana Bastista
#### Encadrante INSA : Maïwenn Pineau

### Installation et import des modules

In [1]:
import io
import os 
import copy
import pandas as pd
import numpy as np
from collections import Counter

## Importation de vcf sample1 :

In [2]:
def check_extension(path):
    """
    Argument :
        
        path : string, chemin vers le fichier à vérifier.
        
        Cette fonction permet de bien vérifier que le fichier d'entré contient
    bien l'extension .vcf.
    
    Attention : sur Windows cela peut poser un problème car les extensions de fichiers 
    ne sont pas toujours apparentes dans les chemins.
    
    Return:
    
        True ou False si le fichier est dans les normes.
    """
    
    split = path.split(".")
    format_name = len(split) -1 

    if (split[format_name].lower() != "vcf"):
        print("Votre fichier n'est pas au bon format, format attendu : vcf")
        return(False)
    else :
        print("Succès : extension vcf détectée")
        return(True)

In [3]:
def check_format(path):
    """
    Argument :
    
        path : string, chemin vers le fichier à vérifier.
        
        Dans tous le fichiers VCF, la première informative doit être de la forme : ##format=VCFv4.x.
    Cette fonction permet de le vérifier, elle renvoie True dans le cas échéant et False sinon.
    
    Return :
        
        True ou False si le fichier est dans les normes.
    """
    with open(path, 'r') as f:
        line = f.readline()
    
    if (line.find("VCF") != -1): #Si on trouve 'VCF' dans line 
        return (True)
    else:
        return(False)

In [4]:
def check_missing_data(df):
    """
    Argument : 
    
        df : dataframe.
    
        Cette fonction permet de vérifier si le fichier vcf n'est pas érroné.
    Pour cela on regarde qu'il ne manque pas de colonne, s'il en manque, la fonction 
    renvoie False avec le nom de la colonne manquante.
    On vérifie ensuite qu'il ne manque pas d'information sur chaque variant  (ou ligne),
    pour cela on regarde qu'il n'y ait pas de NaN. S'il y a trop de NaN, elle return False,
    si le nombre de NaN est faible comparé à la taille du fichier, on supprime les lignes où
    sont localisés les NaN.
    
    Return:
    
        -1 :  le fichier n'est pas bon on le rejette ;
        
        0 : Le fichier est à la limite de l'acceptable (quelques NaN) 
        et peut subir une modification pour traiter les NaN ;
        
        1 : Le fichier est validé.
    """
    
    #Vérifier  qu'il ne manque pas une colonne dans le ficher :
    columns = ['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT']
    for col in columns:
        if (col not in df.columns):
            print("Ce fichier vcf est incomplet")
            print("Il manque la colonne {}".format(col))
            return (-1)
        
    #Vérifier que chaque ligne est bien complète (pas de NaN): 
    NaN_col = df.isna().sum(axis=0)
    NaN_cutoff = 3 #nombre de NaN admissibles par fichier vcf, au dèla fichier rejetté.
    
    if (sum(NaN_col) !=0) : #s'il y a des NaN
        if (sum(NaN_col) <= NaN_cutoff) :
            return(0)
            
        else:
            print("Votre fichier un nombre de données manquantes trop élevé.")
            print("Veuillez fournir un vcf de meilleur qualité.")
            return (-1)
    else:
        print("Contrôle des NaN : Ok")
        return(1)

In [5]:
def drop_NaN_rows(df):
    
    """
    Argument: 
    
        df : dataframe.
        
        Cette fonctionne permet de supprimer les lignes dans lesquelles on aurait des NaN.
    
    Return :
    
        new_df : dataframe dont les lignes contenant des 'NaN' ont été supprimées.
    """
    
    new_df = copy.deepcopy(df) # Pour ne pas ecraser notre df initial 
    NaN_line = df.isna().sum(axis=1)
    indexes = []
    for i, line in enumerate(NaN_line.values):
                if (line != 0):
                    indexes.append(i)
    for idx in indexes :
                new_df = df.drop([idx], axis = 0)
            
    print("Suppression des NaN : Ok")      
    return (new_df)

In [6]:
def quality_control(df, verbose = True):
    """
    Argument:

        df : dataframe à controler.

        Cette fonction fait appel à notre fonction check_missing_data,
    elle fait une synthèse sur la qualité de notre fichier au moment de l'importer
    dans la fonction read_vcf.

    Return :

        df : dataframe avec contrôle qualité effectué.
    """

    miss = check_missing_data(df)
    #print(miss)

    if (miss == 0):
        if(verbose):
            print("contrôle qualité du VCF : Mauvais. \n")
        return (False)

    elif (miss == 1):
        if(verbose):
            print("Contrôle qualité du VCF : Bon. \n")
        return (True)

In [7]:
def read_vcf(path, verbose = True):
    """
    Arguments :

        path : string,  chemin vers le fichier vcf à importer ;

        Cette fonction permet d'importer un fichier vcf et de le convertir en dataframe.
    Il y a la possiblité de faire en amont un contrôle qualité via l'appel de la fonction quality_control
    si QC == True, sinon importation classique.

    Return :

        vcf_df : dataframe du fichier vcf importé.
    """
    if (check_extension(path) == True) :
        if (verbose):
            print("Succès : extension vcf détectée pour le fichier{}".format(path))
            print("\n")

        try:
            if (check_format(path) == True) :
                if (verbose):
                    print("Succès : Format VCF détecté pour le fichier{}".format(path))
                with open(path, 'r') as f:
                    lines = [l for l in f if not l.startswith('##')]

                vcf_df = pd.read_csv( io.StringIO(''.join(lines)), dtype={'#CHROM': str, 'POS': int, 'ID': str,
                                                                       'REF': str, 'ALT': str,'QUAL': str,
                                                                       'FILTER': str, 'INFO': str, 'FORMAT': str},
                                 sep='\t').rename(columns={'#CHROM': 'CHROM'})

                return(vcf_df)

        except IOError :
            if(verbose):
                print("Fichier introuvable. \n")
            return(False)
    else:
        return(False)

## Contrôle des filtres :

In [8]:
def quality_filter(df, reject = ['LowQual', 'INDEL_SPECIFIC_FILTERS;LowQual']):

    """
    Arguments :

        df : dataframe issu d'un vcf ;

        reject : liste des filtres à ne pas garder.


        Cette fonction permet de filtrer (supprimer les lignes) des variants d'un dataframe
    dont le 'FILTER' est contenu dans la liste 'reject'.

    Return :

        new_df : le dataframe filtré.
    """
    new_df = copy.deepcopy(df)

    for muta in df.index: #idem que dans quality_filter_normal, on enleve les filters de mauvaise qualite
        if df["FILTER"][muta] in reject:
            new_df = df.drop(labels = muta, axis=0)

    new_df.index = range(0, len(new_df), 1)  #reajustement des indexes apres le drop
    print("Filtrage des variants de qualité :", *reject, sep='\n')

    return (new_df)

In [9]:
def quality_filter_normal(df_normal, reject = ['LowQual', 'INDEL_SPECIFIC_FILTERS;LowQual'], index = True):
    """
    Arguments :

        df_normal : dataframe issu d'un vcf de tissu sain ;

        reject : liste des filtres à ne pas garder ;

        index : booleen qui indique le revoie ou non d'un indexe.

        Cette fonction permet de filtrer (supprimer les lignes) des variants d'un dataframe issus d'un tissu normal
    dont le 'FILTER' est contenu dans la liste 'reject'.
    Chaque ligne supprimée se retrouve indexée et par son numero de chromosome et par la position du variant
    dans un tuple afin de les supprimer également en aval dans le dataframe tumoral.

    Return :

        df_n : le dataframe normal filtré ;

        indexes : liste de tuples contenant les #chrom et les positions des variants supprimés
            sur le dataframe normal pour appliquer la même opération sur le dataframe tumoral.
    """

    df_n = copy.deepcopy(df_normal) #deep copy pour ne pas ecraser l'original
    indexes = []

    for muta in df_normal.index:
        if df_normal["FILTER"][muta] in reject:

            #On stocke le #CHROM et la #POS correspondant dans un tuple,
            #pour effectuer la même suppression dans notre fichier tumoral apres :
            indexes.append( (df_normal["CHROM"][muta], df_normal["POS"][muta]))

            #On supprime ensuite la ligne correspondante car mauvaise qualite
            df_n = df_normal.drop(labels = muta, axis=0)

    df_n.index = range(0, len(df_n), 1) #reajustement des indexes apres le drop

    if (index == True):
        return(df_n, indexes)
    else:
        return(df_n)

In [10]:
def quality_filter_tumor(df_tumor, index_normal, reject = ['LowQual', 'INDEL_SPECIFIC_FILTERS;LowQual']):
    """
    Arguments:

        df_tumor : dataframe issu d'un vcf de tissu tumoral ;

        index_normal : liste de tuples contenant les #chrom et les positions des variants supprimés
                        pour appliquer la même opération sur le dataframe tumoral;

        reject : liste des filtres à ne pas garder ;

        Cette fonction permet de filtrer un dataframe issu d'un tissu tumoral.
    Dans un premier temps on supprime tous ce qui a été supprimé dans le dataframe normal complémentaire.
    Ensuite on supprime les varaiants dont le 'FILTER' est contenu dans la liste 'reject'

    Return :

        df_t : dataframe tumoral filtré.
    """

    #print(np.unique([index_normal[i][0] for i in range(len(index_normal)) ] ) )

    df_t = copy.deepcopy(df_tumor) #deep copy pour ne pas ecraser l'original

    for chrom_pos in index_normal: #chrom_pos : tuple (#CHROM, #POS)
        chrom = chrom_pos[0] #CHROM
        pos = chrom_pos[1] #POS
        if(pos in df_tumor["POS"].loc[df_tumor["CHROM"] == chrom].values): #on regarde si la position indexee de df_normal est aussi dans df_tumor
            tmp_index = int(np.argwhere(df_tumor["POS"].loc[df_tumor["CHROM"] == chrom].values == pos)) #indexe de cette position
            #print(chrom, pos, tmp_index)
            df_t = df_tumor.drop(labels = tmp_index, axis=0) #on supprime la ligne ou il y a cette position

        if(len(np.unique([index_normal[i][0] for i in range(len(index_normal)) ] )) > 1):
            df_t.index = range(0, len(df_t), 1)

    for muta in df_t.index: #idem que dans quality_filter_normal, on enleve les filters de mauvaise qualite
        if df_tumor["FILTER"][muta] in reject:
            df_t = df_tumor.drop(labels = muta, axis=0)

    df_t.index = range(0, len(df_t), 1)  #reajustement des indexes apres le drop

    return (df_t)

In [11]:
def global_filter(df_tumor, df_normal, reject = ['LowQual', 'INDEL_SPECIFIC_FILTERS;LowQual']):
    
    """
    Arguments :
    
        df_tumor : dataframe issu d'un vcf de tissu tumoral ;
        
        df_normal : dataframe issu d'un vcf de tissu sain ;
        
        reject : liste des filtres à ne pas garder ;
        
        Fonction qui prend en entrée 2 fichiesr vcf complémentaires (tumoral et normal issus d'un même individu),
    et qui fait appel aux fonctions quality_filter_normal et quality_filter_tumor afin de les filtrer,
    selon les filtres retenus dans la liste reject.
    
    Return :
    
        filtered_df_tumor : dataframe tumoral filtré ;
        
        filtered_df_normal : dataframe normal fitré.
            
            
    """
    
    filtered_df_normal = quality_filter_normal(df_normal, reject, index=True)[0]
    indexes2remove = quality_filter_normal(df_normal, reject, index=True)[1]
    
    filtered_df_tumor = quality_filter_tumor(df_tumor, indexes2remove, reject)
    
    print("Filtrage des variants de qualité :", *reject, sep='\n')
    
    return (filtered_df_tumor,
           filtered_df_normal)

In [12]:
sample1_normal = read_vcf("./samples/Sample1_normal_dna.vcf")
sample1_tumor = read_vcf("./samples/Sample1_tumor_dna.vcf")
sample1_somatic = read_vcf("./samples/Sample1_somatic_dna.vcf")

Succès : extension vcf détectée
Succès : extension vcf détectée pour le fichier./samples/Sample1_normal_dna.vcf


Succès : Format VCF détecté pour le fichier./samples/Sample1_normal_dna.vcf
Succès : extension vcf détectée
Succès : extension vcf détectée pour le fichier./samples/Sample1_tumor_dna.vcf


Succès : Format VCF détecté pour le fichier./samples/Sample1_tumor_dna.vcf
Succès : extension vcf détectée
Succès : extension vcf détectée pour le fichier./samples/Sample1_somatic_dna.vcf


Succès : Format VCF détecté pour le fichier./samples/Sample1_somatic_dna.vcf


In [13]:
print(len(sample1_normal), len(sample1_tumor), len(sample1_somatic))

161052 170921 251


## Creation de fichiers 'lite'  (chr1)

In [14]:
def select_chr(df, chrom):
    """
    Arguments :
    
        df : dataframe.
        
        chrom : liste, le ou les chromosomes à conserver.
        
        Cette fonction permet de ne conserver qu'un ou plusieurs chromosomes parmi l'ensemble du dataframe,
    afin de réduire le dataframe et ainsi reduire les temps de chargement pour la suite de ce pipeline
    (ex : calcul du TMB qui peut être très long selon la taille du fichier etc ..)
    
    Return :
    
        new_df : dataframe n'ayant conservé que l'information sur les chromosomes renseignés par l'utilisateur.
    """
    new_df = copy.deepcopy(df)
    
    if (len(chrom) == 1): #Garder un seul chromosome
        new_df = df.loc[df["CHROM"] == str(chrom[0])]
        new_df.index = range(0, len(new_df), 1)  #reajustement des indexes
        return(new_df)
    
    elif (len(chrom) >1): #Conserver plusieurs chromosomes
        chrom = np.unique(sorted(chrom)) # numero chromosome dans l'ordre croissant
        new_df = df.loc[df["CHROM"] == str(chrom[0])]
        for i in range(1, len(chrom), 1):
            tmp_df = df.loc[df["CHROM"] == str(chrom[i])]
            new_df = pd.concat([new_df, tmp_df])
            #print(len(tmp_df), len(new_df))
            
        new_df.index = range(0, len(new_df), 1)
        #print("Seuls les chromosomes suivants ont bien été conservés : ", *chrom, sep = "\n")
        return(new_df)

In [15]:
sample1_normal_test = select_chr(sample1_normal, ['1'])
sample1_tumor_test = select_chr(sample1_tumor, ['1'])
F_sample1_tumor_test,  F_sample1_normal_test= global_filter(sample1_tumor_test, sample1_normal_test)

Filtrage des variants de qualité :
LowQual
INDEL_SPECIFIC_FILTERS;LowQual


In [16]:
Counter(F_sample1_normal_test['FILTER'])

Counter({'PASS': 14021,
         'VQSRTrancheSNP99.90to100.00': 261,
         'LowQual': 860,
         'INDEL_SPECIFIC_FILTERS;LowQual': 125,
         'INDEL_SPECIFIC_FILTERS': 51})

## Test du QC des filtres :

## Quand on a tumor et normal :

In [17]:
def compare(df_tumor, df_normal):
    """
    Arguments :
    
        df_tumor : dataframe issu d'un vcf de tissu tumoral ;
        
        df_normal : dataframe issu d'un vcf de tissu sain ;
    
            On regarde si une mutation présente dans le fichier vcf tumoral est également présente 
        dans le fichier vcf sain (dit normal). Dans le cas échéant on ne compte pas cette mutation 
        comme étant une mutation propre à la tumeur, car elle est peut être localisée à d'autres 
        endroits dans l'organisme (simple SNP par exemple). 
        Dans le cas contraire on peut considèrer cette mutation comme étant somatique est propre à la tumeur.
        
    Return :
    
        indexes : liste d'ID de mutation présentes dans le dataframe tumoral et absente dans le normal.
    """
    
    CHROMS = np.unique(df_tumor['CHROM'].values)
    indexes = []
    
    for _chr_ in CHROMS:
        df_t = df_tumor.loc[df_tumor["CHROM"] == _chr_]
        df_n = df_normal.loc[df_normal["CHROM"] == _chr_]
        
        df_t.index = range(0, len(df_t), 1)
        df_n.index = range(0, len(df_n), 1)
        
        #df to np
        POS_tumor = df_t['POS'].values
        POS_normal = df_n['POS'].values
        
        for muta in df_t.index:
            #print(_chr_)
            NB_ALT_tumor = len(list(df_t['ALT'][muta]))
            START = int(POS_tumor[muta])
            END = START + len(list(df_t['ALT'][muta]))

            #On identifie une mutation par ses positions de début et de fin (start et end)
            if (START in list(POS_normal)): #même debut ?
                # !!une meme position peut sur des chr differents!! 
                index = int(np.argwhere(POS_normal == START))
                if (END == int(POS_normal[index]) + len(list(df_n['ALT'][index]) ) ): #même fin ?
                    pass # non somatique
                else:
                    indexes.append(df_t['ID'][muta])
            else:
                indexes.append(df_t['ID'][muta])
                
    return (indexes)

In [18]:
indexes = compare(F_sample1_tumor_test, F_sample1_normal_test)
len(indexes)

1696

In [19]:
def create_somatic(tumor_path, somatic_path, indexes):
    """
    Arguments:
    
        tumor_path : chemin vers le fichier tumoral vcf ;
        
        somatic_path : chemin vers le futur fichier somatic vcf de sortie;
        
        indexes : indexes : liste d'ID de mutation présentes dans le dataframe tumoral et absente dans le normal. 
        
        Cette fonction permet d'écrire dans un fichier toutes les mutations présentes dans le tissus tumoral,
    et absentes dans le tissu normal. Cette comparaison est faite avec la fonction compare. 
    
    Return :
        
        Rien (0)
        
    """
    
    headers = []
    lines = []

    with open(tumor_path, "r") as f_in:
        for line in f_in:
            if line.startswith('#'):
                headers.append(line)
            elif not line.startswith('#'):
                lines.append(line)

    with open(somatic_path, "w") as f_out:
        for i in range(len(headers)):
                f_out.write(headers[i])
        for j in range(len(lines)-1):
            if (lines[j].split()[2] in indexes):
                f_out.write(lines[j])
                
    return(0)

In [20]:
create_somatic("./samples/Sample1_tumor_dna.vcf", 
               "./Sample1_somatic_test.vcf", indexes)

0

## Annovar / bash to python :

In [21]:
os.system("perl ./annovar/convert2annovar.pl -format vcf4 ./Sample1_somatic_test.vcf > ./Sample1_somatic_test.avinput")

0

In [22]:
#os.system("perl ./annovar/annotate_variation.pl -downdb -buildver hg19 -webfrom annovar refGene ./annovar/humandb/")

In [23]:
os.system("perl ./annovar/annotate_variation.pl -out ./Sample1_somatic_test -build hg19 ./Sample1_somatic_test.avinput ./annovar/humandb/")

0

## Calcul du TMB :

In [24]:
def keep_variants(infile, outfile, synonyme=False, coding=True):
    
    """
    Arguments :
    
        infile : fichier somatic obtenue après annovar ;
        
        outfile : fichier somatic filtré en ne gardant que certaines mutations (choix de l'utilisateur); 
        
        synonyme : booléen, True : on garde les mutation synonymes, sinon False ;
        
        coding : booléen, True : on garde toutes les mutations qui touchent à la protéine finale
                (ex : stop gain, stop loss, frameshift/nonframeshift, insertion/délection), sinon False.
        
        
        Return :
        
            rien (0)
    """
    
    with open(infile, "r") as infile, open(outfile, "w") as outfile:
        for line in infile:
            ligne=line.split()
            if coding==True:
                if synonyme==False and ligne[1]!="synonymous" and ligne[1]!="unknown" :
                    outfile.write(line)
                if synonyme==True and ligne[1]!="unknown" :
                    outfile.write(line)
            if coding==False:
                if synonyme==False and ligne[1]=="nonsynonymous" :
                    outfile.write(line)
                if synonyme==True and (ligne[1]=="nonsynonymous" or ligne[1]=="synonymous") :
                    outfile.write(line)
    return (0)

In [25]:
def TMB_tumor_normal(somatic_infile, somatic_outfile, exome_length = 1, synonyme = False, coding = True):
    """
    Arguments :
    
        somatic_infile : fichier somatique.exonic_variant_function obtenue après annovar ;
        
        somatic_outfile : fichier filtrer après la fonction keep_variants ;
        
        exome_length : int, tailler de l'exome de référence pour calculer un taux.
    
    Return :
    
        TMB : float, Taux de mutation.
    """
    keep_variants(somatic_infile, somatic_outfile, synonyme, coding)
    
    with open(somatic_outfile, 'r') as infile:
        variants = [l for l in infile if not l.startswith('#')]
    
    TMB = len(variants) / exome_length
    return (TMB)

In [26]:
TMB_tumor_normal( somatic_infile= "./Sample1_somatic_test.exonic_variant_function",
             somatic_outfile= "./Sample1_somatic_test.text")

29.0

In [27]:
def TMB_somatic(somatic_infile, exome_length = 1):
    """
    Arguments :
    
        somatic_infile : fichier somatique.exonic_variant_function obtenue après annovar ;
        
        exome_length : int, tailler de l'exome de référence pour calculer un taux.
    
    Return :
    
        TMB : float, Taux de mutation.
    """
    
    with open(somatic_infile, 'r') as infile:
        variants = [l for l in f if not l.startswith('#')]
    
    TMB = len(variants) / exome_length
    return (TMB)

## Quand on a que tumoral :

In [28]:
#os.system("perl ./annovar/convert2annovar.pl -format vcf4 samples/Sample1_tumor_dna.vcf > samples/Sample1_tumor_dna.avinput")

In [29]:
#os.system("perl ./annovar/annotate_variation.pl -downdb -webfrom annovar -build hg19 exac03 annovar/humandb/")

In [30]:
#os.system("perl ./annovar/annotate_variation.pl -filter -build hg19 -dbtype exac03 samples/Sample1_tumor_dna.avinput ./annovar/humandb/")

In [31]:
def TMB_tumor (exac03, exome_length = 1):
    """
    Argument : 
        exac03 : chemin vers fichier texte issus de annovar avec la probabilité pour chaque variant d'être associé 
        à la tumeur ;
        
        exome_length : int, tailler de l'exome de référence pour calculer un taux.
        
    Return : 
        
        TMB : float, Taux de mutation.
    """
    
    TMB = 0
    with open(str(exac03), "r") as exac03:
        for line in exac03:
            line_sp=line.split('\t')
            if float(line_sp[1]) == 0:
                TMB += 1
    return(TMB / exome_length)

In [32]:
TMB_tumor("./samples/Sample1_tumor_dna.avinput.hg19_exac03_dropped")

285.0